In [ ]:
!pip install pyspark

In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: 2dims/
  inflating: 2dims/part-00000        
 extracting: 2dims/_SUCCESS          
 extracting: 2dims/.part-00000.crc   
 extracting: 2dims/._SUCCESS.crc     
  inflating: 2dims/part-00001        
 extracting: 2dims/.part-00001.crc   
   creating: 3dims/
  inflating: 3dims/part-00000        
 extracting: 3dims/_SUCCESS          
 extracting: 3dims/.part-00000.crc   
 extracting: 3dims/._SUCCESS.crc     
  inflating: 3dims/part-00001        
 extracting: 3dims/.part-00001.crc   
   creating: example_test/
  inflating: example_test/part-00000  
 extracting: example_test/_SUCCESS   
 extracting: example_test/.part-00000.crc  
 extracting: example_test/._SUCCESS.crc  
  inflating: example_test/part-00001  
 extracting: example_test/.part-00001.crc  


In [ ]:
test = False

In [ ]:
import pyspark
from pyspark.sql import SparkSession
if test:
  spark = SparkSession.builder \
                      .master("local[*]") \
                      .config("spark.executor.memory", "4g") \
                      .config("spark.driver.memory", "1g") \
                      .appName("mlibs") \
                      .getOrCreate()
else:
  spark = SparkSession.builder \
                    .master("spark://master:7077") \
                    .config("spark.executor.memory", "1g") \
                    .config("spark.driver.memory", "3g") \
                    .appName("mlibs") \
                    .getOrCreate()
sc = spark.sparkContext


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/29 12:12:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
MACHINES_NUMBER = 3

In [ ]:
!hdfs dfs -rm -r -f /input.txt

2023-05-29 12:12:13,280 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted /input.txt


In [ ]:
!ls

 2dims	 3dims	'=0.0.7'   balladyna.txt   data.zip   example_test


In [ ]:
!hdfs dfs -put example_test /input.txt
!hdfs dfs -ls /

t = sc.textFile('hdfs://master:9000/input.txt')
t.take(20)

2023-05-29 12:12:18,425 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-05-29 12:12:22,018 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   3 jw429685 supergroup     199726 2023-03-27 13:39 /balladyna.txt
drwxr-xr-x   - jw429685 supergroup          0 2023-05-29 12:12 /input.txt
drwxr-xr-x   - jw429685 supergroup          0 2023-03-27 13:36 /test


['(1, 5, False)',
 '(3, 8, True)',
 '(4, 2, False)',
 '(5, 9, True)',
 '(6, 7, False)',
 '(7, 3, True)',
 '(8, 1, False)',
 '(9, 4, True)']

In [ ]:
import ast

df = t.map(lambda v: ast.literal_eval(v))
df.take(10)

[(1, 5, False),
 (3, 8, True),
 (4, 2, False),
 (5, 9, True),
 (6, 7, False),
 (7, 3, True),
 (8, 1, False),
 (9, 4, True)]

In [ ]:
import random
import math

In [ ]:
# test generator
if test:
  N = 1_000_000
  VAL_RANGE_MULTIPLIER = 10_000
  DIMS = 2
  x_ys = random.sample(range(N * VAL_RANGE_MULTIPLIER), DIMS * N)
  data = [(*[x_ys[DIMS * i + mod]  for mod in range(DIMS)], random.choice([True, False])) for i in range(N)]
  #data = [(x_ys[2 * i], x_ys[2 * i + 1], random.choice([True, False])) for i in range(N)]
  #data = [(1,5, False), (3,8, True), (4,2, False), (5,9,True), (6,7, False), (7,3, True), (8,1, False), (9,4, True)]
  df = sc.parallelize(data)
  #df.saveAsTextFile("example_test")
  #data_frame = df.toDF()
  #data_frame.write.mode("overwrite").parquet('df_example')
  #input = spark.read.parquet('df_example')
  #input.rdd.collect()

In [ ]:
import bisect
t = MACHINES_NUMBER

def terasort(input, key_fn=lambda x: x, include_ranking=False):
  n = input.count()
  m = n / t
  p = 1 / m / math.log(n*t)
  input = input.map(lambda x: (key_fn(x), x))
  samples = input.sample(False, p)

  sampled_data = samples.collect()

  sample_size = len(sampled_data)

  sampled_data = sorted(sampled_data)
  if len(sampled_data) == 0:
    sampled_data = input.take(1)
  boundaries = [sampled_data[i * sample_size // t] for i in range(1, t)]

  broadcast_boundaries = sc.broadcast(boundaries)

  def get_machine(x):
    return bisect.bisect_left(broadcast_boundaries.value, x)

  inputs_with_machines_ids = input.map(lambda x: (get_machine(x), x))
  input_machine_groups = inputs_with_machines_ids.groupByKey()
  sorted_within_partitions = input_machine_groups.mapValues(lambda lst: sorted(lst))
  if include_ranking:
    machine_counts = sorted(sorted_within_partitions.mapValues(len).collect())
    machine_counts_broadcast = sc.broadcast(machine_counts)
    def count_ranking(machine_id, values):
      rank = sum(machine_count for (m_id, machine_count) in machine_counts_broadcast.value if m_id < machine_id)
      res = []
      for val in values:
        (key_fn_val, value) = val
        res.append((value, rank))
        rank += 1
      return res
    sorted_inputs = sorted_within_partitions.flatMap(lambda machineId_values: count_ranking(machineId_values[0], machineId_values[1]))
  else:
    sorted_inputs = sorted_within_partitions.flatMap(lambda machine_keyval: machine_keyval[1])
    sorted_inputs = sorted_inputs.map(lambda key_val: key_val[1])
  return sorted_inputs

terasort(df, lambda x: x[1], True).take(10)

[((8, 1, False), 0),
 ((4, 2, False), 1),
 ((7, 3, True), 2),
 ((9, 4, True), 3),
 ((1, 5, False), 4),
 ((6, 7, False), 5),
 ((3, 8, True), 6),
 ((5, 9, True), 7)]

In [ ]:
from math import log2, ceil
n = df.count()
DIMS = len(df.take(1)[0]) - 1
logn = ceil(log2(n))
def int_to_binary_string(num):
    return '{0:b}'.format(num).zfill(logn)

def label_dimensions(df):
  for dim in range(DIMS-1):
    df_with_ranking = terasort(df, lambda x: x[dim], True)
    df = df_with_ranking.map(lambda val_rank: (*val_rank[0], int_to_binary_string(val_rank[1])))
  return df.map(lambda xs_q_ls: (*xs_q_ls[:DIMS+1], (xs_q_ls[DIMS+1:]))) # packing labels into single tuple
df_with_labels = label_dimensions(df)
df_with_labels.take(10)

[(1, 5, False, ('000',)),
 (3, 8, True, ('001',)),
 (4, 2, False, ('010',)),
 (7, 3, True, ('101',)),
 (8, 1, False, ('110',)),
 (9, 4, True, ('111',)),
 (5, 9, True, ('011',)),
 (6, 7, False, ('100',))]

In [ ]:
import itertools
def get_prefixes(x_y_isquery_label):
  (*xs, is_query, labels) = x_y_isquery_label
  prefixes = []
  for label in labels:
    dim_prefixes = []
    for idx, c in enumerate(label):
      if (c == '1' and is_query) or (c == '0' and not is_query):
        dim_prefixes.append(label[:idx])
    prefixes.append(dim_prefixes)
  prefixes_combs = itertools.product(*prefixes)
  return [(*xs, is_query, prefixes) for prefixes in prefixes_combs]

df_with_prefixes = df_with_labels.flatMap(get_prefixes)

In [ ]:
sorted_by_prefix = terasort(df_with_prefixes, lambda x_y_isquery_prefix: (x_y_isquery_prefix[-1], x_y_isquery_prefix[-3], x_y_isquery_prefix[-2]), True) # sorting by label, last dim, is_query
per_machine = math.ceil(sorted_by_prefix.count() / t)
sorted_with_machine_id = sorted_by_prefix.map(lambda value_rank: (value_rank[1] // per_machine, value_rank[0]))
sorted_by_prefix_partitions = sorted_with_machine_id.groupByKey()
#sorted_by_prefix_partitions.collect()

In [ ]:
def count_data_within_partition(machine_id, values):
  last_prefix = None
  current_data_count = 0
  res = []
  for (*xs, is_query, prefix) in sorted(values, key=lambda x_y_q_l: (x_y_q_l[-1], x_y_q_l[-3], x_y_q_l[-2])): # sorting by label, last dim, is_query
    if last_prefix != prefix:
      last_prefix = prefix
      current_data_count = 0
    if is_query:
      res.append((*xs, prefix, current_data_count))
    else:
      current_data_count += 1
  res.append((last_prefix, current_data_count, machine_id))
  return res

In [ ]:
counts_within_partitions = sorted_by_prefix_partitions.map(lambda k_v: count_data_within_partition(k_v[0], k_v[1]))
#counts_within_partitions.collect()

In [ ]:
last_prefixes_counts = counts_within_partitions.map(lambda partition: partition[-1]).collect()
prefixes_counts_broadcast = sc.broadcast(last_prefixes_counts)
last_prefixes_counts

[(('0',), 1, 0), (('11',), 1, 2), (('1',), 1, 1)]

In [ ]:
def count_entire_data(lst):
  (last_label, last_count, machine_id) = lst[-1]
  res = []
  for (*xs, prefix, count) in lst[:-1]:
    sum_of_previous_counts = sum([count for (label, count, id) in prefixes_counts_broadcast.value if prefix == label and id < machine_id])
    res.append(((*xs,), count + sum_of_previous_counts))
  return res

In [ ]:
all_counts = counts_within_partitions.flatMap(count_entire_data)
all_counts.take(10)

[((7, 3), 1),
 ((9, 4), 1),
 ((5, 9), 1),
 ((7, 3), 0),
 ((9, 4), 1),
 ((3, 8), 1),
 ((5, 9), 1),
 ((9, 4), 0)]

In [ ]:
aggregated_counts = all_counts.groupByKey()
summed_counts = aggregated_counts.map(lambda k_v: (k_v[0], sum(k_v[1])))
summed_counts.take(10)

[((7, 3), 1), ((5, 9), 2), ((9, 4), 2), ((3, 8), 1)]

In [ ]:
# testing generated data
def brute(dataset):
  res = {}
  for el in dataset:
    if el[-1]: # is_query
      count = 0
      for val in dataset:
        if val[-1]: # also is query
          continue
        for dim in range(DIMS):
          if el[dim] < val[dim]:
            break
        else:
          count += 1
      res[(*el[:-1],)] = count
  return res
counts = brute(df.collect())
are_equal = True
for (xs, count) in summed_counts.collect():
  #print(x, y, count, counts[(x,y)])
  if(count != counts[xs]):
    print("wrong counts:", xs, count, '!=', counts[xs])
    are_equal = False
  counts[xs] = 0
for kv in counts:
  if counts[kv] != 0:
    print("wrong counts:", kv, 0, '!=', counts[kv])
    are_equal = False
if are_equal:
  print("Counts are equal")

Counts are equal


In [ ]:
!zip -r data.zip 2dims 3dims example_test

  adding: 2dims/ (stored 0%)
  adding: 2dims/part-00000 (deflated 62%)
  adding: 2dims/_SUCCESS (stored 0%)
  adding: 2dims/.part-00000.crc (stored 0%)
  adding: 2dims/._SUCCESS.crc (stored 0%)
  adding: 2dims/part-00001 (deflated 62%)
  adding: 2dims/.part-00001.crc (stored 0%)
  adding: 3dims/ (stored 0%)
  adding: 3dims/part-00000 (deflated 59%)
  adding: 3dims/_SUCCESS (stored 0%)
  adding: 3dims/.part-00000.crc (stored 0%)
  adding: 3dims/._SUCCESS.crc (stored 0%)
  adding: 3dims/part-00001 (deflated 59%)
  adding: 3dims/.part-00001.crc (stored 0%)
  adding: example_test/ (stored 0%)
  adding: example_test/part-00000 (deflated 30%)
  adding: example_test/_SUCCESS (stored 0%)
  adding: example_test/.part-00000.crc (stored 0%)
  adding: example_test/._SUCCESS.crc (stored 0%)
  adding: example_test/part-00001 (deflated 30%)
  adding: example_test/.part-00001.crc (stored 0%)


In [ ]:
from google.colab import files
files.download("data.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>